In [ ]:
# Import Javascript Libraries: Execute this cell...
from IPython.display import Javascript

display(Javascript("jupyter_lib.js"))

# Section 9: Kinematics and Newtonian Mechanics 

Goals:
 - Learn about fundamentals of motion. 
 - Learn how to make objects move using Euler's method.
 - Implement code to move the player and simulate gravity.
 
First, before we begin this lesson, you need to make a level to run the game in. Using your code from last class, make a level in the level editor and save it. We'll use that level in the game engine to begin implementing the mechanics for your player.

## Basic Motion: Velocity

To begin, we need to discuss basic motion for an object. One such way to define motion is speed, or velocity. As you may remember, _average_ velocity is defined as.

$$
v_{avg} = \frac{\Delta d}{\Delta t}
$$

Where $\Delta d$ is the distance covered (change in distance), and $\Delta t$ is the time covered (change in time). Velocity is simply a measure of how "fast" an object is moving. Using some basic algebra, we can rearrange this formula to give us one that we can use to update the location of an object in the game. 

$$
\implies v_{avg} = \frac{d_f - d_i}{\Delta t} 
$$

$$
\implies d_f - d_i = v_{avg} \cdot \Delta t
$$

$$
\implies d_f = v_{avg} \cdot \Delta t + d_i
$$

This raises a simple question: How do we compute the average velocity for a given time step? In real life we experience instantaneous velocities, or velocities that occur for an infinitesimal amount of time ($\Delta t$ approaches, or gets really close 0). We experience many of these, producing a velocity curve. Computing the average velocity of this can be challenging.

![Graph of velocity varying within a time step.](images/velocityplot.png)
$$
{\scriptsize \text{How do we determine the average velocity of the above messy curve?}}
$$

The mathematician Euler provided a simple solution to this problem: We don't. Between each time step, we just assume the velocity is constant, and matches the velocity at the beginning of the time step. If our time steps are small enough, it won't matter (at least to the human eye) as the difference between the average and starting velocities will become smaller and smaller. So, to apply this method we simply replace $v_{avg}$ with $v_i$.

$$
d_f \approx v_i \cdot \Delta t + d_i 
$$

### Task 1

Your goal is to implement a constant change in velocity for the player in the x direction, based on the current arrow keys pressed down. (Moving left or right). All this means is that:

$$
v_i = \frac{2}{1000} \text{ or } \frac{-2}{1000}
$$

Depending on the keys pressed. Above means means 2 blocks per second (Note speed of stuff is in blocks per millisecond due to the time step value passed to update being in milliseconds, and game coordinate space units being in blocks).

Hints:
 - This should go in the `update` function.
 - Use `gameState.keysPressed` to check for `"ArrowLeft"` and `"ArrowRight"` values, indicating the arrow keys are pressed. Set your velocity value accordingly.
 - Update `this.x`, or the x location of the player, using the formula we got above.
 - To use the game engine instead of the level editor, replace `element.levelEditor` with `element.makeGame`. This runs the game instead of opening it in the level editor.
 
Bonus:
 - Adjust the velocity to slow down and speed up your character.

In [ ]:
%%javascript
// Copy and paste you game code here, then modify it...

## A Step Up: Acceleration and Gravity

As you may have noticed, objects don't fall at a constant velocity, but rather speed up as the fall further and further. This is due to the fact that things on Earth's surface experience not a constant change in velocity, but rather a constant acceleration due to the force of gravity. The question is, what is acceleration? Well, acceleration is simply a change in velocity.

$$
a_{avg} = \frac{\Delta v}{\Delta t}
$$

The acceleration at Earth's surface due to gravity is $9.8 \frac{m}{s^2}$, but since this is a game, we don't care if our gravity is 100% realistic, so you can use any value (and most games do). 

How do we simulate acceleration? Well, we can apply Euler's method again, simply rearranging and changing the formula to solve for the final velocity given the original velocity. This gives us:

$$
v_f \approx a_i \cdot \Delta t + v_i
$$

Now in our loop, we first update the velocity using the acceleration, and then update the displacement using the new velocity. This means we'll need to store the velocity between steps.

### Task 2

Your goal is to get your player to fall using gravity, in the y direction. To do this, identify a constant gravity value, like:

$$
a_i = 0.1 / 1000
$$

Then implement the following code:
 - Initialize something in the constructor to store the x and y velocities. I suggest using `this._vx` and `this._vy`, as you will have to when we eventually add collision detection to your object.
 - In `update`:
     - Use the formula above to update `this._vy` with the constant acceleration value you selected.
     - Use formula from task 1 to update the y location (`this.y`) of the object given `this._vy`, the current object's velocity.
     
Bonus:
 - The speed of objects doesn't increase forever. Rather, objects reach a terminal velocity, at which they stop falling faster. Implement a cap on the y velocity to keep the object from falling beyond a certain speed.
     
Once your done, you should have an object that falls! But, it will fall right through blocks. This is because we currently don't detect collisions. That will be discovered in the next section.

In [ ]:
%%javascript
// Copy and paste you game code here, then modify it...

## Final Feature: Collision Detection and Correction

As you may have already noticed, the player currently just flies right through objects, because it doesn't implement collision detection. To detect objects colliding in our engine, we will utilize the simplest shape: An axis-aligned box, or hit box. Detecting collisions between hit boxes is extremely simple (we check on each axis, if the starting point of one box is past the end point of another, and if all are true, they aren't colliding). Handling collisions effectively, on the other hand, is not a simple task. Therefore, the game engine provides built-in collision detection you can use. Specifically, by having your object extend `GameCollisionObject` instead of `GameObject`, they will be added to the collision engine.

In the constructor, there are several settings you can configure that change how an object responds to collisions.
 - `this._collisionSides`: An object literal that defaults to `{"left": true, "top": true, "right": true, "bottom": true}`. By setting it to an object with some of the above entries missing, those sides won't be checked for collisions. This allows for one-way platforms, gates, and other items.
 - `this._movable`: Defaults to false. Indicates an object can't be moved, like blocks. If set to true, the velocity of an object must be stored in `this._vx` and `this._vy`. This is because velocities get automatically zeroed for you when you collide with objects in that direction.
 - `this._solid`: Defaults to true. If an object is not solid, collisions will still be detected, but objects that collide with it will move right through the object.
 
Finally, it provides one method your object's can implement (but don't have to).
 - `handleCollision(obj, side)`: Triggered whenever a collision occurs between this and another `GameCollisionObject`. Passes the other object, and the side of the current object that collided with the other object (`"left"`, `"right"`, `"top"`, or `"bottom"`).
 
Rather then implementing your own collision engine, you'll use the above class to handle any needed collisions for you.

### Task 3

Add collisions to the player and blocks. To do this, change the class you are extending. For blocks, the default settings are exactly what you need. For the player, you'll need to set the movable setting to true. 

Bonus: 
 - If you manage to finish it all, why not try implementing jump on pressing the `"ArrowUp"` key? It should set the y velocity at the time of the jump to a high negative value, shooting the object into the air until gravity takes back over.

In [ ]:
%%javascript
// Copy and paste you game code here, then modify it...